<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

## Lab 4.2.2: Feature Selection

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression


%matplotlib inline

### 5. Forward Feature Selection

> Forward Selection: Forward selection is an iterative method in which we start with having no feature in the model. In each iteration, we keep adding the feature which best improves our model till an addition of a new variable does not improve the performance of the model.

Create a Regression model using Forward Feature Selection by looping over all the features adding one at a time until there are no improvements on the prediction metric ( R2  and  AdjustedR2  in this case).

#### 5.1 Load Wine Data & Define Predictor and Target

In [3]:
## Load the wine quality dataset

# Load the wine dataset from csv
wine = pd.read_csv('winequality_merged.csv')

# define the target variable (dependent variable) as y
y = wine['quality']

# Take all columns except target as predictor columns
predictor_columns = [c for c in wine.columns if c != 'quality']
# Load the dataset as a pandas data frame
X = pd.DataFrame(wine, columns = predictor_columns)

In [6]:
## Create training and testing subsets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#### 5.2 Overview of the code below

The external `while` loop goes forever until there are no improvements to the model, which is controlled by the flag `changed` (until is **not** changed).
The inner `for` loop goes over each of the features not yet included in the model and calculates the correlation coefficient. If any model improves on the previous best model then the records are updated.

#### Code variables
- `included`: list of the features (predictors) that were included in the model; starts empty.
- `excluded`: list of features that have **not** been included in the model; starts as the full list of features.
- `best`: dictionary to keep record of the best model found at any stage; starts 'empty'.
- `model`: object of class LinearRegression, with default values for all parameters.

#### Methods of the `LinearRegression` object to investigate
- `fit()`
- `fit.score()`

#### Adjusted $R^2$ formula
$$Adjusted \; R^2 = 1 - { (1 - R^2) (n - 1)  \over n - k - 1 }$$

#### Linear Regression [reference](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression)

In [15]:
## Flag intermediate output

show_steps = True   # for testing/debugging
#show_steps = False  # without showing steps

In [16]:
## Use Forward Feature Selection to pick a good model

# start with no predictors
included = []
# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
lr = LinearRegression() # create a model for Linear Regression
# get the number of cases in the test data
n = X_test.shape[0]

while True:
    changed = False
    
    if show_steps:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps:
            print(f'(Step) Trying {new_column}...')
            print(f'(Step) - Features = {", ".join(included + [new_column])}')

        # fit the model with the Training data
        fit = lr.fit(X_train[included + [new_column]], y_train) # fit a model; consider which predictors should be included
        # calculate the score (R^2 for Regression)
        r2 = lr.score(X_test[included + [new_column]], y_test) # calculate the score
        # number of predictors in this model
        k = len(included) + 1
        # calculate the adjusted R^2
        adjusted_r2 = 1-(1-r2)*(n-1)/(n-k-1)      # calculate the Adjusted R^2

        if show_steps:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed:
        # update control details
        included.append(best['feature'])
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
        (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break



(Step) Excluded = total sulfur dioxide, density, citric acid, chlorides, residual sugar, pH, red_wine, free sulfur dioxide, alcohol, fixed acidity, volatile acidity, sulphates
(Step) Trying total sulfur dioxide...
(Step) - Features = total sulfur dioxide
(Step) - Adjusted R^2: This = 0.001; Best = 0.000
(Step) - New Best!   : Feature = total sulfur dioxide; R^2 = 0.001; Adjusted R^2 = 0.001
(Step) Trying density...
(Step) - Features = density
(Step) - Adjusted R^2: This = 0.091; Best = 0.001
(Step) - New Best!   : Feature = density; R^2 = 0.092; Adjusted R^2 = 0.091
(Step) Trying citric acid...
(Step) - Features = citric acid
(Step) - Adjusted R^2: This = -0.004; Best = 0.091
(Step) Trying chlorides...
(Step) - Features = chlorides
(Step) - Adjusted R^2: This = 0.045; Best = 0.091
(Step) Trying residual sugar...
(Step) - Features = residual sugar
(Step) - Adjusted R^2: This = -0.002; Best = 0.091
(Step) Trying pH...
(Step) - Features = pH
(Step) - Adjusted R^2: This = -0.002; Best = 0

(Step) - Adjusted R^2: This = 0.290; Best = 0.290
(Step) Trying free sulfur dioxide...
(Step) - Features = alcohol, volatile acidity, red_wine, residual sugar, density, sulphates, chlorides, pH, free sulfur dioxide
(Step) - Adjusted R^2: This = 0.289; Best = 0.290
(Step) Trying citric acid...
(Step) - Features = alcohol, volatile acidity, red_wine, residual sugar, density, sulphates, chlorides, pH, citric acid
(Step) - Adjusted R^2: This = 0.289; Best = 0.290
(Step) Trying fixed acidity...
(Step) - Features = alcohol, volatile acidity, red_wine, residual sugar, density, sulphates, chlorides, pH, fixed acidity
(Step) - Adjusted R^2: This = 0.293; Best = 0.290
(Step) - New Best!   : Feature = fixed acidity; R^2 = 0.297; Adjusted R^2 = 0.293
Added feature fixed acidity with R^2 = 0.297 and adjusted R^2 = 0.293

(Step) Excluded = total sulfur dioxide, free sulfur dioxide, citric acid
(Step) Trying total sulfur dioxide...
(Step) - Features = alcohol, volatile acidity, red_wine, residual sug

In [18]:
show_steps = False

In [19]:
included = []
# keep track of model and parameters
best = {'feature': '', 'r2': 0, 'a_r2': 0}
# create a model object to hold the modelling parameters
lr = LinearRegression() # create a model for Linear Regression
# get the number of cases in the test data
n = X_test.shape[0]

while True:
    changed = False
    
    if show_steps:
        print('') 

    # list the features to be evaluated
    excluded = list(set(X.columns) - set(included))
    
    if show_steps:
        print('(Step) Excluded = %s' % ', '.join(excluded))  

    # for each remaining feature to be evaluated
    for new_column in excluded:
        
        if show_steps:
            print(f'(Step) Trying {new_column}...')
            print(f'(Step) - Features = {", ".join(included + [new_column])}')

        # fit the model with the Training data
        fit = lr.fit(X_train[included + [new_column]], y_train) # fit a model; consider which predictors should be included
        # calculate the score (R^2 for Regression)
        r2 = lr.score(X_test[included + [new_column]], y_test) # calculate the score
        # number of predictors in this model
        k = len(included) + 1
        # calculate the adjusted R^2
        adjusted_r2 = 1-(1-r2)*(n-1)/(n-k-1)      # calculate the Adjusted R^2

        if show_steps:
            print('(Step) - Adjusted R^2: This = %.3f; Best = %.3f' % 
                  (adjusted_r2, best['a_r2']))

        # if model improves
        if adjusted_r2 > best['a_r2']:
            # record new parameters
            best = {'feature': new_column, 'r2': r2, 'a_r2': adjusted_r2}
            # flag that found a better model
            changed = True
            if show_steps:
                print('(Step) - New Best!   : Feature = %s; R^2 = %.3f; Adjusted R^2 = %.3f' % 
                      (best['feature'], best['r2'], best['a_r2']))
    # END for

    # if found a better model after testing all remaining features
    if changed:
        # update control details
        included.append(best['feature'])
        excluded = list(set(excluded) - set(best['feature']))
        print('Added feature %-4s with R^2 = %.3f and adjusted R^2 = %.3f' % 
        (best['feature'], best['r2'], best['a_r2']))
    else:
        # terminate if no better model
        break


Added feature alcohol with R^2 = 0.195 and adjusted R^2 = 0.195
Added feature volatile acidity with R^2 = 0.253 and adjusted R^2 = 0.252
Added feature red_wine with R^2 = 0.264 and adjusted R^2 = 0.263
Added feature residual sugar with R^2 = 0.280 and adjusted R^2 = 0.277
Added feature density with R^2 = 0.284 and adjusted R^2 = 0.281
Added feature sulphates with R^2 = 0.290 and adjusted R^2 = 0.287
Added feature chlorides with R^2 = 0.293 and adjusted R^2 = 0.289
Added feature pH   with R^2 = 0.294 and adjusted R^2 = 0.290
Added feature fixed acidity with R^2 = 0.297 and adjusted R^2 = 0.293




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



